## LLM
- the idea to experiment the same methods on multiple LLMs to make sure the result are LLM agnostic, ideally. Practically measure the std b/w different models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

llm_4o = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
llm_4o_mini = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

/opt/homebrew/Caskroom/miniforge/base/envs/ragas/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
langchain_embeddings = LangchainEmbeddingsWrapper(embeddings=embeddings) # any langchain Embeddings instance


In [5]:
from langchain_aws import ChatBedrockConverse
from langchain_aws import BedrockEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

config = {
    "credentials_profile_name": "default",  # E.g "default"
    "region_name": "us-east-1",  # E.g. "us-east-1"
    "llm": "anthropic.claude-3-haiku-20240307-v1:0",  # E.g "anthropic.claude-3-5-sonnet-20240620-v1:0"
}

bedrock_llm = ChatBedrockConverse(
    credentials_profile_name=config["credentials_profile_name"],
    region_name=config["region_name"],
    base_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
    model=config["llm"],
)

bedrock_llm = LangchainLLMWrapper(bedrock_llm)

## tracing

In [366]:
import os
os.environ["LANGCHAIN_PROJECT"]= "Alignment"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Evaluation

In [397]:
from ragas.metrics._aspect_critic import AspectCriticWithReference
from ragas import EvaluationDataset
from datasets import load_dataset, Dataset
import numpy as np

In [398]:
def prepare_eval_dataset(path,postive_clas=0.6, seed=42):
    samples = []
    np.random.seed(seed)
    dataset = Dataset.from_json(path)
    num = int(len(dataset)*postive_clas)
    positive_sample_indices = np.random.randint(0,50,num).tolist()
    negative_sample_indices = [i for i in range(len(dataset)) if i not in positive_sample_indices]
    for i,row in enumerate(dataset):
        dic = {
                "user_input":row["user_input"],
                "reference": row["reference"],
                                
            }
        if i in positive_sample_indices:
            dic.update(
                {
                    "response": row["response"],
                    "target": 1
                }
            )
        else:
             dic.update(
                {
                    "response": row["errored_response"],
                    "target": 0
                }
            )
        samples.append(dic)
    return Dataset.from_list(samples)
            
            
    

In [399]:
dataset = prepare_eval_dataset("datasets/dataset_v4.json",postive_clas=0.6)

In [400]:
len(dataset)

50

In [401]:
y_true = dataset["target"]

In [402]:
eval_dataset = EvaluationDataset.from_hf_dataset(dataset)

## Aspect Critic

In [403]:
def random_examples(n=3):
    from ragas.metrics._aspect_critic import AspectCriticInputWithReference,AspectCriticOutputWithReference
    examples = [(
    AspectCriticInputWithReference(
        user_input="What is the main ingredient in traditional Japanese miso soup?",
        response="The main ingredient in traditional Japanese miso soup is soybeans.",
        reference="The main ingredient in traditional Japanese miso soup is miso paste, which is made from fermented soybeans.",
        criteria="Does the response accurately convey the main ingredient of the soup?",
    ),
    AspectCriticOutputWithReference(
        reason="The response mentions soybeans, which is part of the ingredient but does not fully capture the main ingredient, miso paste.",
        verdict=0,
    ),
),
     (
    AspectCriticInputWithReference(
        user_input="When was the Declaration of Independence signed?",
        response="The Declaration of Independence was signed in 1776.",
        reference="The Declaration of Independence was signed on July 4, 1776.",
        criteria="Is the response factually complete, including both month and year?",
    ),
    AspectCriticOutputWithReference(
        reason="The response provides the correct year but omits the specific date, July 4, making it incomplete.",
        verdict=0,
    ),
),
     (
    AspectCriticInputWithReference(
        user_input="What is the main ingredient in traditional Japanese miso soup?",
        response="The main ingredient in traditional Japanese miso soup is soybeans.",
        reference="The main ingredient in traditional Japanese miso soup is miso paste, which is made from fermented soybeans.",
        criteria="Does the response accurately convey the main ingredient of the soup?",
    ),
    AspectCriticOutputWithReference(
        reason="The response mentions soybeans, which is part of the ingredient but does not fully capture the main ingredient, miso paste.",
        verdict=0,
    ),
)]
    return examples[:n]
     

In [404]:
new_prompt = "Please evaluate the provided responses based on their accuracy and relevance to the user inputs and references. Assign a verdict of 1 for responses that align closely with the reference, even if they contain some additional context or elaboration, and 0 for incorrect responses that contain factual inaccuracies or significant deviations from the reference. Ensure that all critical details from the reference, including specific phrases or terms essential for accuracy, are included in the response to avoid missing key information that could affect the verdict. Provide a brief reason for your verdict, highlighting both strengths and weaknesses in the responses, and ensure that the evaluation remains concise, focusing on core factual accuracy and relevance to the reference. For guidance, examples of significant deviations include discrepancies in key dates or misinterpretations of critical facts"

In [405]:
critic = AspectCriticWithReference(name="answer correctness",
                      definition=new_prompt,dynamic_retrevial=True)


In [406]:
critic.single_turn_prompt.examples = []

In [415]:
critic.llm = llm_4o_mini
critic.embedding_model = langchain_embeddings

In [416]:
from ragas import evaluate

In [417]:
result = evaluate(dataset=eval_dataset[:],metrics=[critic])

Evaluating: 100%|█████████| 50/50 [00:16<00:00,  3.00it/s]


In [418]:
df = result.to_pandas()

In [419]:
df["target"] = y_true
df.head()

,user_input,response,reference,answer correctness,target
0,How did the invention of the wheel impact anci...,# The Revolutionary Impact of the Wheel on Anc...,The invention of the wheel was a pivotal momen...,1,0
1,How did the discovery of fire impact early hum...,# The Transformative Power of Fire in Early Hu...,The discovery of fire was a pivotal moment in ...,1,1
2,What were the major impacts of the Agricultura...,# The Transformative Impacts of the Agricultur...,"The Agricultural Revolution, which began aroun...",1,1
3,What were the key events and consequences of t...,# The Fall of Constantinople: A Turning Point ...,"The Fall of Constantinople occurred on May 29,...",0,0
4,How did the birth of democracy in Athens shape...,# The Birth of Democracy in Athens: A Revoluti...,"The birth of democracy in Athens, around the 5...",0,0


In [420]:
y_pred = df["answer correctness"].values.tolist()

## evaluate the evaluation

In [421]:
from sklearn.metrics import f1_score,precision_score,recall_score

In [422]:
print("f1",f1_score(y_true,y_pred))
print("precision",precision_score(y_true,y_pred))
print("recall",recall_score(y_true,y_pred))

f1 0.8301886792452831
precision 0.7096774193548387
recall 1.0


## Analysis

In [8]:
def get_confusion_indices(y_true, y_pred):
    """
    Function to find the indices of TP, FP, FN, and TN.

    Parameters:
    y_true (list or np.array): Ground truth binary labels (0 or 1).
    y_pred (list or np.array): Predicted binary labels (0 or 1).

    Returns:
    dict: A dictionary containing lists of indices for TP, FP, FN, and TN.
    """
    tp = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true == 1 and pred == 1]
    fp = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true == 0 and pred == 1]
    fn = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true == 1 and pred == 0]
    tn = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true == 0 and pred == 0]

    return {
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn
    }

In [26]:
confusion_matrix = get_confusion_indices(y_true,y_pred)

In [126]:
from typing import Any
def serialize_for_json(data: Any) -> Any:
    """
    Convert custom objects into a JSON-serializable format.

    Parameters:
    data (Any): The data to be converted, which may contain custom objects.

    Returns:
    Any: A JSON-serializable version of the input data.
    """
    if isinstance(data, dict):
        return {key: serialize_for_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [serialize_for_json(item) for item in data]
    elif hasattr(data, "__dict__"):
        # Convert custom objects by serializing their attributes (assumes they use __dict__)
        return serialize_for_json(data.__dict__)
    else:
        return data  # Return the data as-is if it is already JSON-serializable


In [28]:
from collections import defaultdict
samples = defaultdict(list)
traces = result.traces
for qdrant,indices in confusion_matrix.items():
    for idx in indices:
        samples[qdrant].append(serialize_for_json(traces[idx]['answer correctness']))

In [29]:
[len(val) for val in samples.values()]

[17, 7, 1, 19]

In [30]:
import json
with open("datasets/dataset_v4_training_aspect_critic_annotated.json","w") as file:
    json.dump(samples,file)

## Embed and store samples

In [119]:
import json
import numpy as np
samples = json.load(open("datasets/dataset_v4_training_aspect_critic_annotated.json"))
                     
# embeddings.aembed_documents()

In [37]:
def embed(num_samples=20,seed=42):
    tp = samples["TP"]
    tn = samples["TN"]
    fp = samples['FP']
    fn = samples['FN']
    all_samples = tp + tn + fp + fn
    final_samples = []
    for sample in all_samples:
        if sample in tp:
            qdrant = 'TP'
        elif sample in fp:
            qdrant = 'FP'
        elif sample in tn:
            qdrant = 'TN'
        else:
            qdrant = 'FN'
        sample = list(sample.values())[0]
        sample['qdrant'] = qdrant
        final_samples.append(sample)

    np.random.seed(seed)
    selected_samples = list(np.random.choice(final_samples,size=num_samples))
    embed_points = [
        "\n".join(list(data['input'].values()))
        for data in selected_samples
    ]
    vectors = embeddings.embed_documents(embed_points)
    return selected_samples,np.array(vectors)

In [364]:
def json_save(data,path):
    with open(path,"w") as file:
        json.dump(data,file)

In [39]:
vector_samples, vectors = embed(num_samples=20)


In [135]:
from collections import Counter
Counter([sample['qdrant'] for sample in vector_samples]).most_common()

[('TN', 9), ('TP', 6), ('FP', 4), ('FN', 1)]

In [41]:
np.save('indices/selected_20_indices_input_resp_ref.npy', vectors)
json_save(vector_samples,'indices/selected_20_indices_input_resp_ref.json')

In [11]:
data = json.load(open("indices/selected_20_indices.json"))

In [77]:
def add_target(data):
    new_data = []
    for item in data:
        if item['qdrant'] in ['TP','FN']:
            item['target'] = 1
        elif item['qdrant'] in ['TN','FP']:
            item['target'] = 0
        new_data.append(item)
    return new_data

data = add_target(data)
        
    

### Test retrival

In [31]:
from ragas.metrics._aspect_critic import AspectCriticWithReference
critic = AspectCriticWithReference(name="answer correctness",
                      definition="Given the user_input, reference and response. Is the response correct compared with the reference",)

th

In [32]:
critic.embedding_model = langchain_embeddings
critic.llm = llm_4o_mini

In [33]:
from ragas import SingleTurnSample
dataset = Dataset.from_json("datasets/dataset_v4.json")
eval_sample = SingleTurnSample(**dataset[0])
eval_sample.user_input = "What were the causes and consequences of the Chernobyl Disaster?"

In [34]:
await critic.single_turn_ascore(eval_sample)

0

## prompt optimisation

In [96]:
from sampling import SingleExample,PromptFromCorrectExample
from ragas import evaluate

In [97]:
from ragas.metrics._aspect_critic import AspectCriticInputWithReference, AspectCriticOutputWithReference


In [98]:
def format_dict(dic):
    string = ""
    for key, val in dic.items():
        string += f"\n{key}:\n\t{val}\n"
    return string

async def reverse_engineer_instruction_from_correct_examples(data,llm,num_instructions=5, num_samples=10,seed=42):
    """
    reverse engineer prompts using correct example
    """
    generated_prompts = []
    
    np.random.seed(seed=seed)
    correct_examples = [sample for sample in data if sample['qdrant'] in ['TN','TP']]

    prompt = PromptFromCorrectExample()
    for i in tqdm(range(num_instructions)):
        input_list, output_list = [],[]
        
        examples  = np.random.choice(correct_examples,size=3)
        curr_examples = []
        for example in examples:
            _ = example['input'].pop('criteria') if 'criteria' in example['input'] else None
            curr_examples.append((format_dict(example["input"]),example['output'][0]))
        prompt_input = SingleExample(examples=curr_examples)
        response = await prompt.generate(data=prompt_input,llm=llm)
        generated_prompts.append(response.instruction)

    return generated_prompts


    

In [99]:
candidate_prompts=await reverse_engineer_instruction_from_correct_examples(data,llm_4o_mini,num_samples=5)

100%|███████████████████████| 5/5 [00:12<00:00,  2.53s/it]


In [114]:
from datasets import Dataset
import random
from collections import defaultdict

def stratified_sample_with_fallback(dataset: Dataset, batch_size: int, target_column: str = 'target') -> Dataset:
    """
    Function to sample a batch of data with as equal distribution of target values as possible.
    
    Parameters:
        dataset (Dataset): The Hugging Face dataset to sample from.
        batch_size (int): The number of samples in the output batch.
        target_column (str): The column name containing the target values.
        
    Returns:
        Dataset: A batch of data with as close to equal distribution of target values as possible.
    """
    np.random.seed(seed=42)
    # Get all unique target values and count samples per target
    target_values = dataset.unique(target_column)
    target_counts = {target: 0 for target in target_values}
    for example in dataset:
        target_counts[example[target_column]] += 1
    
    # Calculate initial number of samples per class
    num_classes = len(target_values)
    samples_per_class = batch_size // num_classes
    remaining_slots = batch_size

    # Track sampled indices
    sampled_indices = []
    
    # Sampling loop with fallback for insufficient samples
    for target in target_values:
        available_indices = [idx for idx, example in enumerate(dataset) if example[target_column] == target]
        num_to_sample = min(samples_per_class, len(available_indices))
        
        # Sample available indices
        sampled_indices.extend(random.sample(available_indices, num_to_sample))
        remaining_slots -= num_to_sample
    
    # Handle remaining slots by resampling from classes with available samples
    while remaining_slots > 0:
        for target in target_values:
            if remaining_slots <= 0:
                break
            available_indices = [idx for idx, example in enumerate(dataset) if example[target_column] == target]
            if available_indices:
                # Sample one additional index from available samples
                sampled_indices.append(random.choice(available_indices))
                remaining_slots -= 1

    # Shuffle sampled indices to ensure randomness
    random.shuffle(sampled_indices)
    return dataset.select(sampled_indices)

In [165]:
def score_prompts(dataset,candidate_prompts,batch_size,metric_kwargs={}):
    result_scores = []
    for prompt in candidate_prompts:
        critic = AspectCriticWithReference(name="answer correctness",
                          definition=prompt,**metric_kwargs)
        dataset = stratified_sample_with_fallback(dataset,batch_size,target_column='qdrant')
        y_true = dataset['target']
        eval_dataset = [data['input'] for data in dataset]
        eval_dataset = EvaluationDataset.from_list(eval_dataset)
        result = evaluate(dataset=eval_dataset,metrics=[critic])
        traces = result.traces
        df = result.to_pandas()
        y_pred = df["answer correctness"].values.tolist()
        incorrect_indices = [i for i in range(len(y_pred)) if y_pred[i]!=y_true[i]]
        feedback_samples = []
        for idx in incorrect_indices:
            if dataset[idx]['qdrant'] in ['TP','TN']:
                dic ={
                    "input":dataset[idx]["input"],
                    "incorrect_output":serialize_for_json(traces[idx]['answer correctness']),
                    "expected_output":dataset[idx]["output"][0]
                }
                dic['incorrect_output'] = dic['incorrect_output']['0_single_turn_aspect_critic_prompt_with_reference']['output'][0]
                feedback_samples.append(dic)
        fscore = f1_score(y_true,y_pred)
        result_scores.append({"prompt":prompt,"score":fscore,"feedback":feedback_samples})
    return result_scores
        

In [168]:
training_dataset = Dataset.from_list(data)
metric_kwargs = {
    "llm":llm_4o_mini,
    "embedding_model":embeddings,
    "dynamic_retrevial": False
}
prompt_scores = score_prompts(training_dataset,candidate_prompts[:],10,metric_kwargs)

Evaluating: 100%|█████████| 10/10 [00:03<00:00,  2.90it/s]
Flattening the indices: 100%|█| 10/10 [00:00<00:00, 2198.8
Evaluating: 100%|█████████| 10/10 [00:02<00:00,  4.09it/s]
Flattening the indices: 100%|█| 10/10 [00:00<00:00, 2354.5
Evaluating: 100%|█████████| 10/10 [00:03<00:00,  2.57it/s]
Flattening the indices: 100%|█| 10/10 [00:00<00:00, 2256.7
Evaluating: 100%|█████████| 10/10 [00:04<00:00,  2.42it/s]
Flattening the indices: 100%|█| 10/10 [00:00<00:00, 1755.3
Evaluating: 100%|█████████| 10/10 [00:03<00:00,  2.90it/s]


In [236]:
from sampling import FeedbackExample, FeedbackMutationInput, FeedbackMutationPrompt

In [237]:
async def get_feedback_for_prompt(prompt,llm):
    mutation_prompt = FeedbackMutationPrompt()
    original_prompt = prompt['prompt']
    examples = []
    for item in prompt['feedback']:
        input_ = item['input']
        if 'criteria' in input_:
            _ = input_.pop("criteria")
        input_ = format_dict(input_)
        prompt_input = FeedbackExample(input=input_,output=item["incorrect_output"],expected_output=item["expected_output"])
        examples.append(prompt_input)
    prompt_input = FeedbackMutationInput(prompt=original_prompt,examples=examples)
    response = await mutation_prompt.generate(data=prompt_input,llm=llm)
    return response.feedbacks
        

In [238]:
feedbacks = await get_feedback_for_prompt(prompt_scores[0],llm_4o_mini)

In [239]:
feedbacks

['Clarify the criteria for assigning a verdict of 1 or 0, ensuring that the prompt explicitly states that a verdict of 1 should be given for responses that align closely with the reference, even if they contain some additional context or elaboration.',
 'Emphasize the importance of including all critical details from the reference in the response, such as specific phrases or terms that are essential for accuracy, to avoid missing key information that could affect the verdict.',
 'Encourage a more concise evaluation of responses, suggesting that while additional context can be informative, it should not detract from the core factual accuracy and relevance to the reference.']

In [240]:
from sampling import FeedbackMutationPromptGeneration,FeedbackMutationPromptInput

In [241]:
async def generate_prompt_from_feedback(prompt,feedbacks,llm):
    improvement_prompt = FeedbackMutationPromptGeneration()
    prompt_input = FeedbackMutationPromptInput(prompt=prompt,feedbacks=feedbacks)
    response = await improvement_prompt.generate(data=prompt_input,llm=llm)
    return response.instruction

In [242]:
new_prompt = await generate_prompt_from_feedback(prompt_scores[0]['prompt'],feedbacks,llm_4o_mini)

In [247]:
prompt_scores = score_prompts(training_dataset,[new_prompt],10,metric_kwargs)

Evaluating: 100%|█████████| 10/10 [00:04<00:00,  2.18it/s]


In [249]:
prompt_scores

[{'prompt': 'Please evaluate the provided responses based on their accuracy and relevance to the user inputs and references. Assign a verdict of 1 for responses that align closely with the reference, even if they contain some additional context or elaboration, and 0 for incorrect responses that contain factual inaccuracies or significant deviations from the reference. Ensure that all critical details from the reference, including specific phrases or terms essential for accuracy, are included in the response to avoid missing key information that could affect the verdict. Provide a brief reason for your verdict, highlighting both strengths and weaknesses in the responses, and ensure that the evaluation remains concise, focusing on core factual accuracy and relevance to the reference. For guidance, examples of significant deviations include discrepancies in key dates or misinterpretations of critical facts.',
  'score': 0.7272727272727273,
  'feedback': []}]

In [360]:
data = json.load(open("indices/selected_20_indices_input_resp_ref.json"))

In [361]:
_ = [item['input'].pop('criteria') for item in data]

In [365]:
json_save(data,'indices/selected_20_indices_input_resp_ref.json')